<h1 align= 'center' style='color:#ff8c1a'> Predicting the price of the stock and Calculating profit using linear Regression 📈 📉 📈 📉 </h1>

<h3 style='color:#9999ff'> Index: </h3>

-   import required libraries  
    
    - Library for data manupulation
    - Library for Linear Algebra
    - libraries for Data Visulization


-   Getting / Collecting Data
    
    - load dataset/csv file 
 

-   Understanding of Data
    
    - Find shape of dataset
    - Collecting information about each feature
    - Getting columns / feature name
 
 
-    Ensuring columns are the right type
    - Convert Data feature type to datetime



-    Cheking null values
    - Find count of null values
    - Find Percentage of null values
    - Check Visually
    
    
    
-    Perform EDA using pandas_profiling    
    - Overview
    - Variables
    - Interactions
    - Correlations
    - Missing values
    - Sample
    
    
-   Perform Prediction 
     - Define model parameters
     - Create lag function
     - Splitting dataset into train and test
     - Data visualization using plotly


- Model Building
    - Importing required libraries
    - Apply Linear Regression
        - Create Object 
        - Fit model
        - Prediction
        - Calculate MSE
        - Calculate R2
        - print 
    - Apply ANN Regression
        - Create Object 
        - Perfor StandardScaler
        - Fit model
        - Prediction
        - Calculate MSE
        - Calculate R2
        - print 
        
        
- Function Calling


- Calculating Profit


- Thank you!

<h2 style='color:#c61aff'>- Import required libraries

In [ ]:
# library for handling multiple csv file
import glob

# library for data manupulation
import pandas as pd

# library for mathematical opration
import numpy as np

#libraries for data visulization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import missingno as mn
%matplotlib inline

<h2 style='color:#c61aff'>- Getting data

In [ ]:
#name of all csv files are stored in 'all_files' variable
#all_files = [i for i in glob.glob('*.{}'.format('csv'))]
#all_files

In [ ]:
#combine all files in the list
#combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])

#export to csv
#combined_csv.to_csv( "stock_data.csv", index=False, encoding='utf-8-sig')

In [ ]:
# Here, I used Alphabet Inc.stock.csv
data = pd.read_csv('../input/mncs-stock-dataset-for-the-last-20-years/Alphabet Inc.stock.csv')
data.head()

<h2 style='color:#c61aff'> 
- Understand Data

*Find shape of dataset*

In [ ]:
print('Dataframe having {} features and {} entries'.format(data.shape[1],data.shape[0]))

*Collecting information about each feature*

In [ ]:
data.info()

- From above we can say that :
    - Dataset having 8 feature and 55742 entries.
    - ['Date','Company'] this features are object type.
    - ['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close'] this features are float type.
    - There is no null value are present in our dataset.

*Features name in our dataset*

In [ ]:
print('Features name in our dataset : ',list(data.columns[0:]))

<h2 style='color:#c61aff'>- Ensuring columns are the right type

*In dataset Data feature is of object type insted of datatime so we chnage it to datatime formate*

In [ ]:
date_format = '%Y-%m-%d'
data['Date'] = pd.to_datetime(data['Date'],format=date_format)

In [ ]:
data.info()

Now,Our Data is in perfect format

<h2 style='color:#c61aff'>- Check Null Values

In [ ]:
# find count of nall values
data.isnull().sum()

In [ ]:
#find percentage of null values
data.isnull().mean()

In [ ]:
mn.matrix(data,color=(.3, .7, .9))
plt.xlabel("Feature",fontsize=25,color='#0099cc')
plt.ylabel("Count",fontsize=25,color='#0099cc')
plt.title(" Cheking Null values in each feature Visully",fontsize=30,color='#007399')

<h2 style='color:#c61aff'>- Perform EDA using pandas_profiling

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
auto_eda = ProfileReport(data, title="Performing EDA on 20 year's MNC's stock dataset" ,explorative=True)

In [ ]:
ProfileReport(data)

<h2 style='color:#c61aff'> - Perform Prediction

#### Define model parameters

In [ ]:
# Shifts is a day in which you want to predict price of stock
shifts = [1,5,10]

#perform training and testing data
train_per = 0.75 #70%

#plotting dimention
w = 16 #d
h = 6 #height

In [ ]:
#create lag function

#which is help us to move close

def createlags(data,lag_size):
    shift = -lag_size
    data['Close_lag']=data['Close'].shift(shift)
    return data,shift

In [ ]:
#function to split data in training and testing
def SplitData(data,train_per,shift):
    train_pt = int(len(data)*train_per)
    
    train_data = data.drop(['Company'],axis=1).iloc[:train_pt,:]
    test_data = data.drop(['Company'],axis=1).iloc[train_pt:,:]
    
    x_train = train_data.iloc[:shift,1:-1]
    y_train = train_data['Close_lag'][:shift]
    x_test = test_data.iloc[:shift,1:-1]
    y_test = test_data['Close'][:shift]
    
    print(len(x_train),len(y_train),len(x_test),len(y_test)) 
    return x_train,y_train,x_test,y_test,train_data,test_data



In [ ]:
# Making visulization helper function
def visulization_plotly(train_data,test_data,pred,ticker,w,h,shift_days,name):
    #this is parameters for scatter plot
    plot1 = go.Scatter(x = train_data.index,y = train_data['Close'], name = 'Actual Training Data') # train data
    plot2 = go.Scatter(x = test_data.index,y = test_data['Close'], name = 'Actual Testing Data') # test Data
    plot3 = go.Scatter(x = test_data.index,y = pred, name = 'Predicted Data') # Prediction    
    
    # this line set parameters
    line = { 'data':[plot1,plot2,plot3],
            'layout':
                {
                    'xaxis':{'title':'Date'},
                    'yaxis':{'title':'$'},
                    'title' : name + ' - ' + ticker + ' - ' +str(shift_days)
                }          
            }
    fig = go.Figure(line)
    fig.show()

<h2 style='color:#c61aff'>- Model Building

In [ ]:
# import libraries for model building

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

*create function of each*

In [ ]:
# linear regression
def LinearReg_fun(x_train,y_train,x_test,y_test):
    print("Linear Regression ")
    
    #regression model
    lr = LinearRegression()
    print("object created")
    
    #fit model
    lr.fit(x_train,y_train)
    print("model fit")
    
    #predict
    lr_pred = lr.predict(x_test)
    print("value predicted")
    
    #calculate MSE
    lr_mse = mean_squared_error(y_test,lr_pred)
    
    #calculate R2
    lr_r2 = lr.score(x_test,y_test)
    
    print('MSE is : {} \n R2 is : {}'.format(lr_mse,lr_r2))
    return lr_pred

In [ ]:
# ANN regression
def ANN_fun(x_train,y_train,x_test,y_test):
    print("ANN Regression")
    
    #perfor standardscaler
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    #regression model
    mlp = MLPRegressor(activation='identity',random_state=1,max_iter=1000)
    
    #fit model
    mlp.fit(x_train_scaled,y_train)
    
    #predict
    mlp_pred = mlp.predict(x_test_scaled)
    
    #calculate MSE
    mlp_mse = mean_squared_error(y_test,lr_pred)
    
    #calculate R2
    mlp_r2 = mlp.score(x_test_scaled,y_test)
    
    print('MSE is : {} \n R2 is : {}'.format(mlp_mse,mlp_r2))
    return mlp_pred

<h2 style='color:#c61aff'>- Function calling

In [ ]:
# Call function
ticker = 'GOOG'
for j in shifts:
    print(str(j)+' -days out prediction \n -----------------------------------')
    
    # create lags
    data_lag,shift = createlags(data,j)
    
    #split the data
    x_train,y_train,x_test,y_test,train_data,test_data = SplitData(data_lag,train_per,shift)
    print("Preprocessing is done")
    
    #Linear Regression
    lr_pred = LinearReg_fun(x_train,y_train,x_test,y_test)
    visulization_plotly(train_data,test_data,lr_pred,ticker,w,h,j,'Linear Regression')
    
    #ANN Regression
    mlp_pred = ANN_fun(x_train,y_train,x_test,y_test)
    visulization_plotly(train_data,test_data,lr_pred,ticker,w,h,j,'ANN Regression')

<h2 style='color:#c61aff'>  
    - Calculting Profit

In [ ]:
def calprofit(test_data):
    pd.set_option('mode.chained_assignment',None)
    test_data['pred'] = np.nan
    test_data['pred'].iloc[:-j] = lr_pred
    
    test_data['Change'] = test_data['Close_lag']-test_data['Close']
    test_data['Change_pred'] = test_data['Close_lag'] - test_data['Close']
    test_data['Made_Money?'] = np.where(test_data['Change_pred']/test_data['Change']>0,1,-1)
    test_data['Profit'] = np.abs(test_data['Change_pred'])*test_data['Made_Money?']
    
    profit = test_data['Profit'].sum()
    days = len(test_data['Made_Money?']==1)
    
    return "- Would have made $ {}, Percentage of days we made money : {}".format(profit,days)

In [ ]:
calprofit(test_data)

<h1 align='center' style='color:#80ff80'>  Thank you !